# 3.1. 生成课程内容

## 🚄 前言
将大模型应用于课程制作流程，这是一个非常时髦的技术问题。但是面对陌生的科技主题、不同年龄的学生，我们如何能快速开发出科普课程？本节课程介绍如何利用文本生成模型 Qwen-Max 和图像生成模型 Flux-Merged，为不同年龄的学生自动生成定制化的图文并茂的科普课程脚本。


## 🍁 课程目标
学完本课程后，你将能够：
* 了解 Qwen-Max，并使用 Qwen-Max 生成文本
* 了解 Flux-Merged，并使用 Flux-Merged 生成图像


## 📖 课程目录

- [1. 原理介绍](#🧮-1-原理介绍)
- [2. 代码实践](#🛠️-2-代码实践)
    - [2.1. 环境准备](#21-环境准备)
    - [2.2. 设置 API 客户端](#22-设置-api-客户端)
    - [2.3. 生成课程脚本](#23-生成课程脚本)
    - [2.4. 生成插画提示词](#24-生成插画提示词)
    - [2.5. 生成插画](#25-生成插画)
    - [2.6. 合并课程和插图](#26-合并课程和插图)
  
  
## 🧮 1. 原理介绍

本次课程用到了以下模型：

* [Qwen-Max](https://bailian.console.aliyun.com/#/model-market/detail/qwen-max)：通义千问2.5系列千亿级别超大规模语言模型，支持中文、英文等不同语言输入。随着模型的升级，qwen-max将滚动更新升级。如果希望使用固定版本，请使用历史快照版本。

* [Flux-Merged](https://bailian.console.aliyun.com/#/model-market/detail/flux-merged)：FLUX.1-merged模型结合了"DEV"在开发阶段探索的深度特性和"Schnell"所代表的高速执行优势。通过这一举措，FLUX.1-merged不仅提升了模型的性能界限，还拓宽了其应用范围。

结合使用这两种模型生成课程内容的过程如下：

<div align="center">
    <img src="https://gw.alicdn.com/imgextra/i3/O1CN01hj66zt1lp8eTpytmx_!!6000000004867-0-tps-2408-142.jpg" alt="流程" width="90%"/>
</div>


## 🛠️ 2. 代码实践

接下来，让我们执行以下代码让大模型生成一节课程内容。

### 2.1. 环境准备

1. 安装 Python 库。

In [1]:
! pip install -r requirements.txt -q

2. 导入必要的模块。

In [2]:
import os
import openai
from typing import Union
import requests
import json
from http import HTTPStatus
from urllib.parse import urlparse, unquote
from pathlib import PurePosixPath
from dashscope import ImageSynthesis
import dashscope
from utils import create_directory, save_file,load_config

3. 设置环境变量

In [15]:
import os
import sys
sys.path.append("../")
from config.load_key import load_key
load_key()
print(os.environ["DASHSCOPE_API_KEY"])

4. 加载配置文件。

In [11]:
# 加载配置文件代码在 utils.load_config
project_config = load_config("config.json")

### 2.2. 设置 API 客户端
设置 OpenAI 的 API 客户端，用于后续调用阿里云百炼的 Qwen-Max 模型和 Flux-Merged 模型。

In [12]:
client = openai.OpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

### 2.3. 生成课程脚本

首先，我们让文本生成模型 Qwen-Max 扮演课程讲师，来生成课程脚本。

1. 定义一个 `create_course_script` 函数，用于使用 Qwen-Max 根据给定的主题生成一份适合特定年龄段的课程脚本。它结合了不同的解释级别、创造性和幽默感，以调整课程内容的风格，使其更符合目标受众的需求。

In [13]:
def create_course_script(topic_name, level_of_explanation, age, creativity_level, humour_level):
    """
    根据主题、解释水平、年龄以及创意和幽默水平生成对应的课程脚本。

    :param topic_name: str，课程主题的名称。
    :param level_of_explanation: str，决定解释的复杂程度，值可以为 '初学者', '中级', '高级'。
    :param age: int，授课对象的年龄。
    :param creativity_level: int，创意的级别（1-10），决定讲解中使用创造性元素的程度。
    :param humour_level: int，幽默的级别（1-10），决定讲解中包含幽默的程度。

    :return: str，生成的课程脚本内容，格式化为Markdown。
    """
    
    # 确保年龄为字符串类型，以便用作提示中的描述
    age_str = str(age)
    
    # 定义各级别的解释方式
    explanation_levels = {
        '初学者': "用简单易懂的方式解释，帮助读者理解直觉、逻辑和重要性。",
        '中级': "以更复杂和深入的方式解释，假设读者有一些先前知识和理解。",
        '高级': "深入讲解主题细节，假设读者有扎实基础，熟悉中级概念。"
    }
    
    # 获取对应级别的解释方式，若未提供则默认为'初学者'
    level_string = explanation_levels.get(level_of_explanation, explanation_levels['初学者'])
    
    # 确定创意和幽默的描述
    creativity_string = (
        "" if creativity_level < 4 
        else "在解释时富有创意，以便于理解。" if creativity_level < 7 
        else "在解释时富有创意并使用类比。"
    )
    
    humour_string = (
        "" if humour_level < 4 
        else "在解释时带一点幽默。" if humour_level < 7 
        else "在解释时增添笑话，使学习更有趣。"
    )
    
    # 构造提示词，综合所有参数生成课程内容的描述
    prompt = (
        f"请生成一个关于{topic_name}的课程内容。"
        f"授课对象为：{age_str}岁的学生。"
        f"解释关键概念。"
        f"课程内容要求：{level_string}{creativity_string}{humour_string}。"
        f"输出应为Markdown格式。"
        f"只生成 # 和 ## 两层目录结构，其他都作为要点内容"
        f"除了课程内容本身，不要输出其他内容"
    )
    
    # 直接调用 API 获取课程脚本
    completion = client.chat.completions.create(
        model="qwen-max-latest",
        messages=[
            {"role": "system", "content": "你是一名专注于科学普及的课程讲师，负责为学生提供清晰的课程内容"},
            {"role": "user", "content": prompt},
        ],
    )
    
    # 解析 API 返回结果，提取生成的课程脚本内容
    dumped_json = json.loads(completion.model_dump_json())
    generated_content = dumped_json['choices'][0]['message']['content']
    print("生成的课程脚本：" + generated_content)  # 打印生成的课程脚本
    
    # 返回生成的课程脚本内容
    return generated_content

2. 调用 `create_course_script` 函数根据 `config.json` 中的配置生成课程脚本。

In [14]:
# 读取配置文件中的课程内容信息
title = project_config["title"]
level_of_explanation = project_config["level_of_explanation"]
age = project_config["age"]
creativity_level = project_config["creativity_level"]
humour_level = project_config["humour_level"]

# 调用函数生成课程脚本
script = create_course_script(title, level_of_explanation, age, creativity_level, humour_level)

# 读取配置文件中的课程脚本路径
course_file_path = project_config["course_file_path"].format(title=project_config["title"])

# 调用函数保存课程脚本
save_file(script, course_file_path)

生成的课程脚本：# 云计算课程

## 介绍
- 欢迎来到神奇的“云”世界！在这里，我们不是谈论天气预报中的云朵，而是指那些让我们的数字生活变得更加轻松的技术。
- 今天，我们将一起探索云计算的概念、类型以及它如何改变着我们日常生活的方方面面。想象一下，如果有一片云可以随身携带，里面装满了你所有的游戏、音乐和电影——这就是云计算的魅力所在！

## 什么是云计算？
- **定义**：简单来说，云计算就是通过互联网访问计算资源（如服务器、存储空间等）。你可以把它看作是一种超级强大的远程计算机服务。
- **历史背景**：虽然听起来很新潮，但其实这个概念早在上世纪60年代就已经被提出了。那时候的人们梦想着有一天能够实现“随时随地获取信息”的愿景。
- **工作原理**：当你使用云服务时，实际上是在利用位于世界各地的数据中心提供的强大计算能力。这就好比借用了别人的超级电脑来完成自己的任务。

## 云计算的主要类型
- **公有云**：面向公众开放的服务平台，任何人都可以通过网络注册并使用其提供的资源。就像一个大型游乐场，每个人都可以进去玩。
- **私有云**：专为特定组织或企业设计的封闭式环境。相当于在家里建了一个小型游乐园，只有家庭成员才能进入享受乐趣。
- **混合云**：结合了公有云与私有云的优势，允许用户根据需要灵活切换。就像是既拥有自家后院的小型游泳池，又能在夏天去海边畅游一样方便。

## 为什么我们需要云计算？
- **成本效益**：无需购买昂贵的硬件设备，按需付费即可获得所需资源。
- **灵活性高**：可以根据业务需求快速扩展或缩减服务规模。
- **易于协作**：多人可以同时在线编辑同一份文档，非常适合团队合作项目。
- **数据安全**：专业的云服务商通常会采取多种措施保护用户数据不被盗取或损坏。

## 云计算的应用实例
- **在线办公软件**：比如Google Docs，可以让多位同事同时修改同一个文件。
- **视频流媒体服务**：Netflix这样的平台就是依靠云端技术向用户提供海量影视内容。
- **智能家居系统**：通过连接至云端服务器，家里的各种智能设备能够实现远程控制等功能。
- **游戏行业**：现在有许多游戏支持云游戏模式，玩家无需下载安装包就能直接在浏览器中体验高质量的游戏。

## 结语
- 云计算正以前所未有的速度发展

### 2.4. 生成插画提示词

接下来，我们让 Qwen-Max 模型扮演插画提示词生成专家，为我们的课程脚本生成适配的插画提示词。

1. 定义一个 `create_illustration_prompt ` 函数，用于根据给定主题生成适合微课程的多幅插图详细信息。该函数使用 Qwen-Max 模型，通过结构化提示词指导模型，确保插图在主题、描述、场景和风格上精准而富有创意。

In [16]:
def create_illustration_prompt(script):
    """
    根据给定的脚本生成插图提示，并通过 API 获取详细插图信息。

    :param script: str，微课程的主题或内容脚本，用于生成插图提示。
    
    :return: str，生成的插图提示，包含详细的插图信息和格式的 JSON 字符串。
    """
    
    # 系统消息，指定模型的角色和任务
    system_message = (
        "您是一位插图提示生成专家，专注于为微课程生成详细的插图提示。"
    )
    
    # 构建插图生成提示
    prompt = f"""生成关于 {script} 的插图。返回仅包含多个插图详细信息的 JSON 数组。
    ### 改进指南:
    - **主题:** 中心概念。
    - **描述:** 详细叙述重点元素、情感和氛围。
    - **场景:** 特定环境（如自然、城市、太空），包括颜色、光线和情绪。
    - **对象:** 主要主题和特征（如人、动物、物体）。
    - **动作:** 对象的动态（如飞行、跳跃、闲逛）。
    - **风格:** 艺术技巧（如抽象、超现实主义、水彩、矢量）。
    - **细节:** 其他特定信息（如纹理、背景元素）。
    ### 生成的提示结构:
    “描述, 场景, 包含对象, 动作. 以风格呈现, 强调细节。”
    ### 提示示例
    [
        {{
            "插图编号": 1,
            "插图位置": "第一课",
            "图片标题": "阳光明媚的日子",
            "插图描述": "一幅富有创意的数字艺术作品，描绘了一只由埃菲尔铁塔构建的长颈鹿。"
        }},
        {{
            "插图编号": 2,
            "插图位置": "第二课",
            "图片标题": "繁星之夜",
            "插图描述": "一幅黑暗幻想肖像，呈现了一匹马奔跑在风暴中，背景火焰般的景观。"
        }}
    ]
    输出格式为JSON。不包含任何额外的文字、解释或评论。"""
    
    # 调用 API 获取插图数据
    completion = client.chat.completions.create(
        model="qwen-max-latest",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt},
        ],
    )
    
    # 解析 API 返回结果
    dumped_json = json.loads(completion.model_dump_json())
    generated_content = dumped_json['choices'][0]['message']['content']  # 提取生成的插图提示内容
    print("生成的插画提示词:", generated_content)  # 打印生成的提示
    
    # 返回生成的插图提示
    return generated_content

2. 调用 `create_illustration_prompt` 函数生成插画提示词。

In [17]:
# 读取配置文件中的插图提示词文件路径
illustrations_prompt_file_path = project_config["illustrations_prompt_file_path"].format(title=project_config["title"])

# 调用函数生成插画提示词
illustrations_prompt = create_illustration_prompt(script)

# 调用函数保存插画提示词
save_file(illustrations_prompt, illustrations_prompt_file_path)

生成的插画提示词: [
    {
        "插图编号": 1,
        "插图位置": "介绍",
        "图片标题": "云中的数字生活",
        "插图描述": "一个充满活力的城市景观，天空中漂浮着各种图标（游戏手柄、音乐符号、电影胶片），象征着云端存储的内容。人们通过手中的平板电脑和手机与这些图标互动。以现代风格呈现，强调色彩鲜艳的背景元素。"
    },
    {
        "插图编号": 2,
        "插图位置": "什么是云计算？-定义",
        "图片标题": "远程计算机服务",
        "插图描述": "一位年轻人正坐在笔记本电脑前工作，屏幕上显示着连接到全球各地服务器的地图。地图上的每个点都用光束相连，形成网络状结构。以矢量风格呈现，突出光线效果。"
    },
    {
        "插图编号": 3,
        "插图位置": "云计算的主要类型-公有云",
        "图片标题": "大型游乐场",
        "插图描述": "一群不同年龄的人们在游乐园里玩耍，每个人手里拿着一部设备访问云端资源。场景设置在一个充满欢乐气氛的主题公园内，使用明亮欢快的颜色。以卡通风格绘制，强调人物表情的生动性。"
    },
    {
        "插图编号": 4,
        "插图位置": "为什么我们需要云计算？-数据安全",
        "图片标题": "保护用户数据",
        "插图描述": "一座坚固的数据中心被多层防护措施包围着，包括防火墙、加密锁等象征性的图像。周围环绕着代表黑客攻击的小怪物试图入侵但未能成功。采用超现实主义手法描绘，重点放在安全机制上。"
    },
    {
        "插图编号": 5,
        "插图位置": "云计算的应用实例-在线办公软件",
        "图片标题": "团队协作编辑文档",
        "插图描述": "几位同事围坐在一张桌子旁，他们面前摆放着各自的电子设备，屏幕上映射出相同的文档界面，显示出多人同时编辑的状态。办公室环境温馨舒适，色调温暖。以写实风格表现，注重人物之间的交流氛围。"
    }
]
目标目录：./output/illust

### 2.5. 生成插画

使用 Flux-Merged 模型为课程生成插画。

1. 定义一个 `generate_illustration ` 函数，用于使用 Flux-Merged 模型根据插图详细信息生成插图。

In [18]:
def generate_illustration(prompt, illustration_number, illustration_title):
    """
    根据给定提示生成插图，并将其保存为文件。

    :param prompt: str，描述生成图像的提示信息。
    :param illustration_number: int，指明插图的顺序编号，用于命名文件。
    :param illustration_title: str，插图的标题，用于命名文件。
    
    :return: str或None，返回生成的图像文件路径；如果生成或保存失败，返回 None。
    """
    
    # 调用生成图像的函数，使用指定的模型并设置生成图像的大小
    rsp = ImageSynthesis.call(model="flux-merged",
                               prompt=prompt,
                               size='1024*1024')
    
    # 创建保存图像的目录
    output_dir = project_config["illustrations_file_folder"]
    os.makedirs(output_dir, exist_ok=True)  # 确保目录存在，以便保存生成的图像

    # 获取项目的标题用于命名文件
    title = project_config["title"]
    
    # 检查生成结果是否为空
    if not rsp.output.results:
        print(f"生成插图失败: {illustration_title}, 返回结果为空")
        return None  # 返回 None，表示生成失败

    # 假设处理第一张生成的插图
    result = rsp.output.results[0]  # 获取生成的第一张图像的结果
    # 构造保存文件的名称
    file_name = f"{title}_{illustration_number}_{illustration_title}.png"
    # 使用指定的输出目录和文件名构建完整的文件路径
    file_path = os.path.join(output_dir, file_name)
    
    # 尝试下载并保存生成的图像
    try:
        with open(file_path, 'wb+') as f:
            f.write(requests.get(result.url).content)  # 从 URL 下载图像内容并写入文件
    except Exception as e:
        print(f"保存图像失败: {file_name}, 错误信息: {e}")
        return None  # 返回 None，表示保存失败
    
    return file_path  # 返回生成的图片文件路径

2. 定义一个 `generate_illustrations` 函数用于处理多个插图的生成。

In [19]:
def generate_illustrations(illustrations):
    """
    生成插图并返回更新后的插图信息的 JSON 格式文本。

    :param illustrations: JSON 字符串或包含插图信息的列表
    :return: 返回更新后的插图信息的 JSON 文本
    """
    output_dir = project_config["illustrations_file_folder"]

    # 检查输入类型
    if isinstance(illustrations, str):
        illustrations = json.loads(illustrations)
    elif not isinstance(illustrations, list):
        raise ValueError("illustrations 需要是一个 JSON 字符串或列表")

    results = []

    # 批量调用生成插图的功能
    for item in illustrations:
        illustration_number = item['插图编号']
        illustration_location = item['插图位置']
        illustration_title = item['图片标题']
        prompt = item['插图描述']

        # 调用生成插图的函数
        image_path = generate_illustration(prompt, illustration_number, illustration_title)

        # 检查生成插图是否成功
        if image_path:  # 如果生成成功
            print(f"生成插图成功: 编号: {illustration_number}, 标题: {illustration_title}, 保存路径: {image_path}")
            # 更新插图路径，直接使用生成的完整路径
            item['插图路径'] = image_path  # 直接赋值生成的路径

            # 将更新后的插图信息添加到结果列表
            results.append(item)
        else:
            print(f"插图生成失败: 编号: {illustration_number}, 标题: {illustration_title}")

    # 将结果列表转换为 JSON 格式的字符串
    final_result = json.dumps(results, ensure_ascii=False, indent=4)  # 确保中文字符正常显示

    print("生成的插图信息:", final_result)
    return final_result  # 返回结果的 JSON 文本

3. 调用 `generate_illustrations` 函数，生成所有插画并将插画信息保存为 JSON 文件。

In [21]:
# 读取配置文件中的课程插画信息文件路径
illustrations_info_file_path = project_config["illustrations_info_file_path"].format(title=project_config["title"])

# 生成插图和插画信息
illustrations_info = generate_illustrations(illustrations_prompt)

# 调用函数保存插画信息
save_file(illustrations_info, illustrations_info_file_path)

### 2.6. 合并课程和插图

最后，我们使用 Qwen-Max 将课程脚本和插画合并，生成一篇图文并茂的科普课程脚本。

1. 定义一个 `add_illustration_to_course_script` 函数，使用 Qwen-Max 将课程脚本和插图合并。

In [15]:
def add_illustration_to_course_script(script, course_script_with_illustrations_file_path, illustrations):
    """
    将插图整合到课程脚本中，并返回带插图的Markdown格式课程脚本。

    参数：
    - script: str，原始课程脚本，包含文本内容。
    - course_script_with_illustrations_file_path: str，保存生成课程脚本的文件路径，用于提供上下文。
    - illustrations: list，插图列表，包含要插入的图像信息。

    返回：
    - str，更新后的带插图的课程脚本，格式为Markdown。
    """
    
    # 创建系统消息，指示助手的角色和任务
    system_message = "你是一名助手，专注于将插画插入课程脚本中。"
    
    # 计算插图目录的路径，以便后续在提示词中使用
    illustrations_dir = "../illustrations/"
    
    # 构建提示词，要求模型在指定位置整合插图
    prompt = (
        "请在相关位置将插图整合到课程脚本中。"
        "输出的带插画的课程脚本保持Markdown格式。"
        f"根据文件路径 {course_script_with_illustrations_file_path}，"
        f"请将插图的相对路径调整为 {illustrations_dir}"
        "。请勿包含任何额外的解释。\n\n"
        f"课程脚本:\n{script}\n\n"
        f"插图列表:\n{illustrations}"
    )
    
    # 调用 API 获取插图数据，与模型进行交互以生成带插图的课程脚本
    completion = client.chat.completions.create(
        model="qwen-max-latest",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt},
        ],
    )
    
    # 解析 API 返回的结果，确保可以读取生成的内容
    dumped_json = json.loads(completion.model_dump_json())
    generated_content = dumped_json['choices'][0]['message']['content']
    print("生成带插画的课程脚本:", generated_content)
    
    # 返回生成的包含插图的课程脚本
    return generated_content

2. 调用 `add_illustration_to_course_script` 函数生成图文并茂的科普课程脚本。

In [22]:
# 读取配置文件中的带插画的课程文件路径
course_script_with_illustrations_file_path = project_config["course_script_with_illustrations_file_path"].format(title=project_config["title"])

# 生成带插图的课程脚本
course_script_with_illustrations = add_illustration_to_course_script(script, course_script_with_illustrations_file_path, illustrations_info)

# 调用函数保存带插图的课程脚本
save_file(course_script_with_illustrations, course_script_with_illustrations_file_path)

## ✅ 本节小结

- 在本次学习和实践中，我们了解了 Qwen-Max 和 Flux-Merged，并使用这些大模型生成了课程内容。
- 这些课程内容得到广泛好评，甚至有机会推广到其他国家的学生，从而引发了国际化的需求。接下来，我们将学习如何使用大模型将课程内容翻译成不同语言，以满足更多学生的需求。

## 🔥 课后小测验

【多选题】3.1.1. 针对如下课程脚本片段：“学习率是深度学习中重要的超参数。” 你希望使用 Qwen-Max 生成一个合适的“文生图”插图提示词，以下哪个提示词最佳？（ ）

A. 学习率

B. 深度学习

C. 一个图表，展示不同学习率对模型训练的影响

D. 超参数

答案：C

解析： 选项 C 提供了清晰的图像描述，包含了图表类型和学习率的核心概念，能够引导“文生图”生成符合要求的插图。其他选项过于简单，缺乏足够的指导信息。